<a href="https://colab.research.google.com/github/dude123studios/SmarterRing/blob/master/Face_Recognition_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
import os
import random
import cv2

Data is from https://www.kaggle.com/havingfun/100-bollywood-celebrity-faces, and includes different celeberty images of about 100~ different classes. We will train our network to check if two different images are of the same class. In this way, we will be able to generalize to any use case of people. We will use the kaggle api to download the dataset. In order to try this yourself, you would need a file called kaggle.json on your computer, which you can get from the kaggle homepage

In [ ]:
from google.colab import files
files.upload() # Browse for the kaggle.json file that you downloaded

# Make directory named kaggle, copy kaggle.json file there, and change the permissions of the file.
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

# You can check if everything's okay by running this command.
! kaggle datasets list

# Download and unzip sign-language-mnist dataset into '/usr/local'
! kaggle datasets download -d havingfun/100-bollywood-celebrity-faces --path '/usr/local' --unzip

Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                       title                                         size  lastUpdated          downloadCount  
--------------------------------------------------------  -------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-wallstreetsbets-posts                       Reddit WallStreetBets Posts                    8MB  2021-02-19 07:57:28           1362  
michau96/restaurant-business-rankings-2020                Restaurant Business Rankings 2020             16KB  2021-01-30 14:20:45           2217  
yorkehead/stock-market-subreddits                         Stock Market Subreddits                        1MB  2021-01-29 13:53:50            241  
aagghh/crypto-telegram-groups                             Crypto telegram groups                       297MB  2021-02-02 09:58:25            159  
timoboz/superbowl-hi

We will use the opencv built in frontal face detector, which will locate the crop of every face in an image. Since the model will be detecting just faces, it is more effecient if we only take the crops the the actors faces

In [ ]:
cascPath=os.path.dirname(cv2.__file__)+"/data/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)
from google.colab.patches import cv2_imshow
def preprocces_img(image_path, np_path):
  img = cv2.imread(image_path)
  #some images in the dataset are non jpg/png type, so we need to filter those out
  try:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  except:
    return None
  #produce image crops of just the face of an actor, becuase we will perform
  #multi instance detection. 
  faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
  if len(faces) == 0:
    return None
  x, y, w, h = faces[0]
  img = img[y:y+h, x:x+w]
  #cv2_imshow(img)
  img = tf.convert_to_tensor(img, dtype=tf.float32)
  img = tf.image.resize(img, (64,64))
  #use mobilenet preproccesing for convinience, even if we use a smaller model
  img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
  numpy_img = img.numpy()
  with open(np_path+'/'+image_path.split('/')[-1].split('.')[0]+'.npy', 'wb') as f:
    np.save(f, numpy_img)
  return img

(CAUTION) This takes a very long time

In [ ]:
!pip install -q tqdm
from tqdm import tqdm

In [ ]:
path1 = '/usr/local/bollywood_celeb_faces_0'
path2 = '/usr/local/bollywood_celeb_faces_1'
path3 = '/usr/local/bollywood_celeb_faces2'
if not os.path.exists('/usr/local/numpy'):
  os.mkdir('/usr/local/numpy')
  os.mkdir('/usr/local/numpy/bollywood_celeb_faces')
#we dont really need to store these, rather, we can use this as a test to see if everything worked
faces = []
for actor in tqdm(os.listdir(path1)):
  if not os.path.exists(numpy_path+'/'+actor):
    os.mkdir(numpy_path+'/'+actor)
  faces.append([preprocces_img(path1+'/'+actor+'/'+actor_img,numpy_path+'/'+actor) for actor_img in os.listdir(os.path.join(path1,actor))])
for actor in tqdm(os.listdir(path2)):
  if not os.path.exists(numpy_path+'/'+actor):
    os.mkdir(numpy_path+'/'+actor)
  faces.append([preprocces_img(path2+'/'+actor+'/'+actor_img,numpy_path+'/'+actor) for actor_img in os.listdir(os.path.join(path2,actor))])
for actor in tqdm(os.listdir(path3)):
  if not os.path.exists(numpy_path+'/'+actor):
    os.mkdir(numpy_path+'/'+actor)
  faces.append([preprocces_img(path3+'/'+actor+'/'+actor_img,numpy_path+'/'+actor) for actor_img in os.listdir(os.path.join(path3,actor))])
for i, person in enumerate(faces):
  faces[i] = [img for img in person if img is not None]

In [ ]:
!rm -rf ./usr/local/bollywood_celeb_faces2
!rm -rf ./usr/local/bollywood_celeb_faces_0
!rm -rf ./usr/local/bollywood_celeb_faces_1

In [ ]:
 !zip -r /usr/local/numpy/bollywood_celeb_faces.zip /usr/local/numpy/bollywood_celeb_faces/

In [ ]:
from google.colab import files
files.download("/usr/local/numpy/bollywood_celeb_faces.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>